In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Bidirectional, LayerNormalization, MultiHeadAttention, Add, Flatten
)

In [15]:
evaluation_results = []
# Modified evaluate_model function to store results in evaluation_results
def evaluate_model(y_true, y_pred, model_name, dataset_type):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    # Append metrics to evaluation_results
    evaluation_results.append({
        'Model': model_name,
        'Dataset': dataset_type,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R²': r2
    })

def create_dataset(dataset, time_step):
    print(f'dataset shape {dataset.shape}')
    dataX, dataY = [], []
    for i in range(len(dataset) - time_step):
        a = dataset[i:(i + time_step), :]  # Features: GHI and Energy delta
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])  # Target: Energy delta
    return np.array(dataX), np.array(dataY)


def data_prep():
    data = pd.read_csv('/Users/yashwanthkaruparthi/Documents/Acads/sem7/design project/execution/data/solar_weather.csv', header=0, infer_datetime_format=True, parse_dates=['Time'], index_col=['Time'])
    
    data = data[data.index.year == 2021]

    # data = data[(data.index.year == 2021)]

    dataset = data[['Energy delta[Wh]', 'GHI']]
    return dataset

In [16]:
time_step = 24

In [17]:
dataset = data_prep()

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)

X, y = create_dataset(scaled_data, time_step)
print(f'X, y shape {X.shape} {y.shape}')
# Split the data into training and testing sets (80% train, 20% test)
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

/var/folders/n5/mlsmdgns0kqfybw1r9qm4n5w0000gp/T/ipykernel_44530/2990087723.py:30: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv('/Users/yashwanthkaruparthi/Documents/Acads/sem7/design project/execution/data/solar_weather copy 2.csv', header=0, infer_datetime_format=True, parse_dates=['Time'], index_col=['Time'])


dataset shape (34368, 2)
X, y shape (34344, 24, 2) (34344,)


In [18]:


# Define the Transformer Encoder Layer
def transformer_encoder(inputs, num_heads, dff, dropout_rate=0.1):
    # Multi-Head Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    # Add & Normalize
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feed Forward Network
    ffn_output = Dense(dff, activation='relu')(attention_output)
    ffn_output = Dense(inputs.shape[-1])(ffn_output)
    ffn_output = Dropout(dropout_rate)(ffn_output)
    # Add & Normalize
    output = Add()([attention_output, ffn_output])
    return LayerNormalization(epsilon=1e-6)(output)

# Model Configuration
time_steps = 24  # Input time steps
features = 2     # Number of features
d_model = 128    # Dimension of model embeddings
num_heads = 4    # Number of attention heads
dff = 512        # Dimension of feed-forward network in Transformer
dropout_rate = 0.2

# Input Layer
inputs = Input(shape=(time_steps, features))

# Bidirectional LSTM Layer
bilstm_output = Bidirectional(LSTM(128, return_sequences=True))(inputs)

# Transformer Encoder Layer
transformer_output = transformer_encoder(bilstm_output, num_heads=num_heads, dff=dff, dropout_rate=dropout_rate)

# Final Layers
flattened = Flatten()(transformer_output)  # Flatten the output
dense1 = Dense(64, activation='relu')(flattened)
dropout = Dropout(0.2)(dense1)
outputs = Dense(1)(dropout)  # Output layer (e.g., regression)

# Build and Compile the Model
model = Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Model Summary
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 24, 2)]              0         []                            
                                                                                                  
 bidirectional_2 (Bidirecti  (None, 24, 256)              134144    ['input_3[0][0]']             
 onal)                                                                                            
                                                                                                  
 multi_head_attention_2 (Mu  (None, 24, 256)              1051904   ['bidirectional_2[0][0]',     
 ltiHeadAttention)                                                   'bidirectional_2[0][0]']     
                                                                                            

In [19]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=1)


Epoch 1/10
415/430 [===========================>..] - ETA: 5s - loss: 0.5855

KeyboardInterrupt: 